In [29]:
import gutenbergpy.textget
import gutenbergpy.gutenbergcachesettings
import pandas as pd 

In [30]:
gutenbergpy.gutenbergcachesettings.GutenbergCacheSettings.TEXT_FILES_CACHE_FOLDER = '../data/texts'

def download_text(id):
    try:
        text = gutenbergpy.textget.get_text_by_id(id)
        text = gutenbergpy.textget.strip_headers(text)
        text = text.decode('utf-8')
        return text
    except Exception as e:
        print("error downloading ID", id)
        raise e

In [31]:
df_metadata = pd.read_csv('../data/gutenberg.csv')
#filter type text
df_metadata = df_metadata[df_metadata['Type'] == 'Text']
#filter langauge english
df_metadata = df_metadata[df_metadata['Language'] == 'en']
#get only Text Authors and Title
df_metadata = df_metadata[['Text#','Authors','Title']]
df_metadata = df_metadata.dropna().reset_index(drop=True)
#remove years
df_metadata["Authors"] = df_metadata["Authors"].str.split(',').str[:-1].str.join(',').str.strip()
df_metadata["Authors"]


/tmp/ipykernel_114691/1996361312.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metadata = pd.read_csv('../data/gutenberg.csv')


0                         Jefferson, Thomas
1                                          
2        Kennedy, John F. (John Fitzgerald)
3                          Lincoln, Abraham
4                                          
                        ...                
55596                   Schiller, Friedrich
55597                        Huneker, James
55598              Flandrau, Charles Macomb
55599         Cornell, Frederick Carruthers
55600                                      
Name: Authors, Length: 55601, dtype: object

Seleccionamos los autores que están en el top 100
https://www.gutenberg.org/browse/scores/top

In [32]:
authors = pd.read_csv('../data/authors100.csv')
#find last '(' and remove everything after
authors['Author'] = authors['Author'].str.split('(').str[:-1].str.join('(').str.strip()

authors


,Author
0,"Shakespeare, William"
1,"Smollett, T. (Tobias)"
2,"Forster, E. M. (Edward Morgan)"
3,"Montgomery, L. M. (Lucy Maud)"
4,"Alcott, Louisa May"
...,...
95,"Balagtas, Francisco"
96,"Marcus Aurelius, Emperor of Rome"
97,"Wodehouse, P. G. (Pelham Grenville)"
98,"Wharton, Edith"


In [33]:
_df_metadata_filtered = df_metadata[df_metadata['Authors'].isin(authors['Author'])]



In [34]:
#pick a random book per author with seed 13
df_metadata_filtered = _df_metadata_filtered.groupby('Authors').sample(n=1, random_state=13).reset_index(drop=True)


In [41]:
len(df_metadata_filtered)

17

In [36]:
_authors_df = pd.read_csv('../data/authors.csv')
_authors_df['Author'] = _authors_df['Author'].str.split('(').str[:-1].str.join('(').str.strip()

#filter only books with authors in the authors.csv
df_metadata_filtered = df_metadata_filtered[df_metadata_filtered['Authors'].isin(_authors_df['Author'])]

In [37]:
code_ids = df_metadata_filtered['Text#'].tolist()
books = [download_text(id) for id in code_ids]

In [38]:
#add books column to dataframe
df_metadata_filtered['Books'] = books

In [39]:
df_metadata_filtered

,Text#,Authors,Title,Books
0,37807,"Alcott, Louisa May",Mountain-Laurel and Maidenhair,"\n\n\n\n\n\n\n\n\n [Illustration: ""Her eyes b..."
1,1212,"Austen, Jane",Love and Freindship [sic],\n\n\n\nLOVE & FREINDSHIP\nAND\nOTHER EARLY WO...
5,9182,"Brontë, Charlotte",Villette,\n\n\n\nVillette\n\nby Charlotte Brontë\n\n\nC...
14,67173,"Christie, Agatha",The Missing Will,\n\n The Missing Wil...
17,1416,"Dickens, Charles",Mrs. Lirriper's Lodgings,\n\n\n\n\nTranscribed from the 1894 Chapman an...
19,40745,"Dostoyevsky, Fyodor",Short Stories,\n\n\n\n\n\n\n SHOR...
20,108,"Doyle, Arthur Conan",The Return of Sherlock Holmes,\n\n\n\ncover\n\n\n\n\nThe Return of Sherlock ...
22,2744,"Dumas, Alexandre",Mary Stuart\r\nCelebrated Crimes,\n\n\n\n\n *MARY S...
34,65214,"Hugo, Victor","Hans of Iceland, Vol. 1 of 2",\n [Illustration]\n...
43,2680,"Marcus Aurelius, Emperor of Rome",Meditations,\n\n\n\nMEDITATIONS\n\nBy Marcus Aurelius\n\n\...


In [40]:
#save dataframe to csv
df_metadata_filtered.to_csv('../data/gutenberg_with_text.csv', index=False)